<a href="https://colab.research.google.com/github/pedro-de-bastos/Capstone/blob/master/Capstone_Notebook_2_Analyzing_Comment_by_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google.cloud
!pip install pystan
import pystan
import pandas as pd
import glob
import pandas as pd
import os
from os import path
import glob
import numpy as np
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import matplotlib.pyplot as plt
import scipy.stats as sts
%matplotlib inline

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/Capstone/instructor-langu-1590688665393-6d06375c82af.json'


  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl


In [ ]:
def response_sentiment_analysis(string_response):
  text_content = string_response

  # Available types: PLAIN_TEXT, HTML
  type_ = enums.Document.Type.PLAIN_TEXT

  # Optional. If not specified, the language is automatically detected.
  # For list of supported languages:
  # https://cloud.google.com/natural-language/docs/languages
  language = "en"
  document = {"content": text_content, "type": type_, "language": language}

  # Available values: NONE, UTF8, UTF16, UTF32
  encoding_type = enums.EncodingType.UTF8

  client = language_v1.LanguageServiceClient()
  annotations = client.analyze_sentiment(document, encoding_type=encoding_type)
  return(annotations)

def get_score(text):
  sa = response_sentiment_analysis(text)
  return sa.document_sentiment.score

def get_magnitude(text):
  sa = response_sentiment_analysis(text)
  return sa.document_sentiment.magnitude

def prep_data(file_path):
  foo = pd.read_csv(file_path)
  foo['Transcripts'] = foo['Transcripts'].astype(str)
  foo['Score'] = foo['Transcripts'].apply(get_score)
  foo['Magnitude'] = foo['Transcripts'].apply(get_magnitude)
  foo['Duration Secs'] = foo['Duration Secs']/1000
  foo['Timestamp'] = foo['Timestamp']/1000
  foo['End'] = foo['End']/1000
  foo['Judgement'] = foo['Score']*foo['Magnitude']
  return foo

def list_sessions():
  sessions = []

  for i in range(1, 11):
    for j in range(1, 3):
      session = 'Class_{}.{}'.format(i, j)
      if session != 'Class_6.1':
        sessions.append(session)
  return sessions

def list_sess_num():
  lst_sess_num = []
  for i in range(1,11):
    for j in range(1, 3):
      if "{}.{}".format(i, j)!="6.1":
        lst_sess_num.append("{}.{}".format(i, j))
  return lst_sess_num

def create_class_to_date_dic(emojis_handraises):
  sessions = list_sessions()

  class_dates = emojis_handraises['Class Date'].unique()
  class_dates = np.flip(class_dates)[0:20]

  dic = {}
  for i, j in zip(sessions, class_dates):
    dic[j] = i
  return dic

def mapper(class_date, emojis_handraises):
  dic = create_class_to_date_dic(emojis_handraises)
  try:
    return dic[class_date]
  except: 
    return np.nan

def prep_emojis(file_path, session, class_start_hour):
  emojis_handraises = pd.read_csv(file_path)
  emojis_handraises['Sessions'] = emojis_handraises['Class Date'].apply(mapper, args=(emojis_handraises,))
  emojis_handraises = emojis_handraises.dropna(axis=0)
  emojis_handraises = emojis_handraises[emojis_handraises['Sessions']=="Class_{}".format(session)]
  emojis_handraises['Start Time'] = pd.to_datetime(emojis_handraises['Start Time'])
  emojis_handraises['Start Time'] = emojis_handraises['Start Time'].apply(adjust_time, args=(class_start_hour,))
  return emojis_handraises

def adjust_time(x, class_start):
  class_start_seconds = (class_start-1)*3600+59*60
  event_seconds = x.hour*3600+x.minute*60+x.second
  return event_seconds - class_start_seconds

def events_counter(end, event_type, emojis):
  emojis = emojis[np.logical_and(emojis['Start Time']>(end-20), emojis['Start Time']<(end+40))]
  emojis = emojis[emojis['Event Data']==event_type]
  return emojis.shape[0]

def merge_data(emojis, foo):
  unique_events = ['{"emotion":"confused"}', '{"emotion":"frown"}', '{"emotion":"hand-raise"}', '{"emotion":"lol"}', '{"emotion":"smile"}', '{"emotion":"snap"}', '{"emotion":"thumbsdown"}']
  for i in unique_events:
    foo[i] = foo['End'].apply(events_counter, args=(i, emojis))
  return foo

def produce_emojis_data(lst):
  datas = {}

  for i, j in zip(lst, list_sess_num()):
    if '/content/drive/MyDrive/Capstone/{}/SliceFoos/foo_{}.csv'.format(lst[0].split('/')[5], j) not in glob.glob(f"/content/drive/MyDrive/Capstone/{lst[0].split('/')[5]}/SliceFoos/*"):
      foo = prep_data(i)
      foo['Session Number'] = "Class_{}".format(j)
      emojis = prep_emojis(f"/content/drive/MyDrive/Capstone/{lst[0].split('/')[5]}/Emoji and Hand Raises/emoji_hands.csv", j, 11)
      datas[j] = merge_data(emojis, foo)
      foo.to_csv('/content/drive/MyDrive/Capstone/{}/SliceFoos/foo_{}.csv'.format(lst[0].split('/')[5], j))
    else:
      emojis = prep_emojis(f"/content/drive/MyDrive/Capstone/{lst[0].split('/')[5]}/Emoji and Hand Raises/emoji_hands.csv", j, 11)
      foo = pd.read_csv('/content/drive/MyDrive/Capstone/{}/SliceFoos/foo_{}.csv'.format(lst[0].split('/')[5], j))
      datas[j] = merge_data(emojis, foo)
  return datas

def produce_and_merge(lst):
  csvs = produce_emojis_data(lst)
  csvs_lst = []
  for i in list_sess_num():
    csvs_lst.append(csvs[i])
  concat_csvs = pd.concat(csvs_lst)
  concat_csvs['Engagement'] = concat_csvs['{"emotion":"confused"}']+concat_csvs['{"emotion":"frown"}']+concat_csvs['{"emotion":"hand-raise"}']+concat_csvs['{"emotion":"lol"}']+concat_csvs['{"emotion":"smile"}']+concat_csvs['{"emotion":"snap"}']+concat_csvs['{"emotion":"thumbsdown"}']
  return concat_csvs

def emotion_lst():
  return ['{"emotion":"confused"}', '{"emotion":"frown"}', '{"emotion":"hand-raise"}', '{"emotion":"lol"}', '{"emotion":"smile"}', '{"emotion":"snap"}', '{"emotion":"thumbsdown"}']

def plot_emotions(concat_data):
  emotions = ['{"emotion":"confused"}', '{"emotion":"frown"}', '{"emotion":"hand-raise"}', '{"emotion":"lol"}', '{"emotion":"smile"}', '{"emotion":"snap"}', '{"emotion":"thumbsdown"}']
  fig, axs = plt.subplots(nrows = 3, ncols=3, figsize=(30, 10))
  #fig.suptitle('Emotions and Occurences')
  colors = ['red', 'green', 'blue', 'orange', 'purple', 'yellow', 'brown']

  emot = 0
  for i in range(3):
    for j in range(3):
      if emot != 7:
        X = np.asarray(concat_data['Judgement']).reshape(-1, 1)
        y = concat_data[emotions[emot]]
        axs[i][j].set_xlabel("Sentiment Score", fontsize=20)
        axs[i][j].set_ylabel('Emotion Occurences', fontsize=20)
        axs[i][j].set_title(emotions[emot], fontsize=20)
        axs[i][j].scatter(X, y, color = colors[emot])
        axs[i][j].tick_params(axis="y", labelsize=20)
        axs[i][j].tick_params(axis="x", labelsize=20)
        emot += 1
  axs[2][1].set_xlabel("Sentiment Score", fontsize=20)
  axs[2][1].set_ylabel('Emotion Occurences', fontsize=20)
  axs[2][1].set_title(emotions[6], fontsize=20)
  axs[2][1].scatter(X, y, color = colors[6])
  axs[2][1].tick_params(axis="y", labelsize=20)
  axs[2][1].tick_params(axis="x", labelsize=20)
  fig.delaxes(axs[2][2])
  fig.delaxes(axs[2][0])
  fig.subplots_adjust(top = 0.9, bottom = -0.7, wspace=0.3, hspace=0.4)
  plt.show()

def hists_2d(concat_data):
  bin = 15
  emotions = ['{"emotion":"confused"}', '{"emotion":"frown"}', '{"emotion":"hand-raise"}', '{"emotion":"lol"}', '{"emotion":"smile"}', '{"emotion":"snap"}', '{"emotion":"thumbsdown"}']
  fig, axs = plt.subplots(nrows = 3, ncols=3, figsize=(30, 10))
  #fig.suptitle('Emotions and Occurences')
  colors = ['red', 'green', 'blue', 'orange', 'purple', 'yellow', 'brown']

  emot = 0
  for i in range(3):
    for j in range(3):
      if emot != 7:
        X = concat_data['Judgement']
        y = concat_data[emotions[emot]]
        axs[i][j].set_xlabel("Sentiment Score", fontsize=20)
        axs[i][j].set_ylabel('Emotion Occurences', fontsize=20)
        axs[i][j].set_title(emotions[emot], fontsize=20)
        axs[i][j].hist2d(X, y, bins=[bin, bin], density=True)
        axs[i][j].tick_params(axis="y", labelsize=20)
        axs[i][j].tick_params(axis="x", labelsize=20)
        emot += 1
  axs[2][1].set_xlabel("Sentiment Score", fontsize=20)
  axs[2][1].set_ylabel('Emotion Occurences', fontsize=20)
  axs[2][1].set_title(emotions[6], fontsize=20)
  axs[2][1].hist2d(X, y, bins=[bin, bin], density=True)
  axs[2][1].tick_params(axis="y", labelsize=20)
  axs[2][1].tick_params(axis="x", labelsize=20)
  fig.delaxes(axs[2][2])
  fig.delaxes(axs[2][0])
  fig.subplots_adjust(top = 0.9, bottom = -0.7, wspace=0.3, hspace=0.4)
  plt.show()

def run_chi2(concat, emoji):
  concat[emoji]
  upper = concat['Judgement'].mean()+concat['Judgement'].std()
  lower = concat['Judgement'].mean()-concat['Judgement'].std()

  positive = concat[concat['Judgement']>upper]
  negative = concat[concat['Judgement']<lower]
  neutral = concat[np.logical_and(concat['Judgement']<upper, concat['Judgement']>lower)]

  positive.high = sum(positive[emoji]>3)
  positive.low = sum(positive[emoji]<=3)

  neutral.high = sum(neutral[emoji]>3)
  neutral.low = sum(neutral[emoji]<=3)

  negative.high = sum(negative[emoji]>3)
  negative.low = sum(negative[emoji]<=3)

  matrix = np.asarray([[neutral.high, neutral.low], [positive.high, positive.low], [negative.high, negative.low]])
  test = sts.chi2_contingency(matrix)
  return test, matrix

def create_pos_neut_neg(concat):   
  positive_thresh = concat_mcallister['Judgement'].mean()+concat_mcallister['Judgement'].std()
  negative_thresh = concat_mcallister['Judgement'].mean()-concat_mcallister['Judgement'].std()

  positive = concat_mcallister[concat_mcallister['Judgement']>positive_thresh]
  negative = concat_mcallister[concat_mcallister['Judgement']<negative_thresh]
  neutral = concat_mcallister[(concat_mcallister['Judgement']>negative_thresh)&(concat_mcallister['Judgement']<positive_thresh)]

  return positive, neutral, negative

def stat_diff_pos_neut_neg(concat):
  positive, neutral, negative = create_pos_neut_neg(concat)
  results = pd.DataFrame(columns=['Number of Comments', 'Mean Engagement', 'Independence T-Test with Positive', 'Independence T-Test with Neutral', 'Independence T-Test with Negative'])

  results['Number of Comments'] = pd.Series([i.shape[0] for i in [positive, neutral, negative]])
  results['Mean Engagement'] = pd.Series([i['Engagement'].mean() for i in [positive, neutral, negative]])
  results['Independence T-Test with Positive'] = pd.Series([sts.ttest_ind(i['Engagement'], positive['Engagement']).pvalue for i in [positive, neutral, negative]])
  results['Independence T-Test with Neutral'] = pd.Series([sts.ttest_ind(i['Engagement'], neutral['Engagement']).pvalue for i in [positive, neutral, negative]])
  results['Independence T-Test with Negative'] = pd.Series([sts.ttest_ind(i['Engagement'], negative['Engagement']).pvalue for i in [positive, neutral, negative]])
  results.index = ['Positive', 'Neutral', 'Negative']
  return results


stan_code_1 = """


data {  
    int<lower=0> n;  //number of comments
    real<lower=0> comments[n]; //the number of engagements
    real<lower=0> alpha;
    real<lower=0> beta; 
}


parameters {
    real<lower=0,upper=60> lambda; //60 was arbitrary, I thought it is big enough
                              
}


model {
    lambda ~ gamma(alpha, beta);  // prior over lambda
    for(i in 1:n)
      comments[i] ~ exponential(lambda); //likelihood
}

"""

def compile_evaluate_stan(stan_code, concat):
  stan_model = pystan.StanModel(model_code=stan_code) #Compiling the model
  _, neutral, _ = create_neg_pos_neut(concat)
  neutral = neutral['Engagement']
  data_dict = {"n":len(neutral), "comments":neutral, "alpha":2, "beta":2}
  stan_results = stan_model.sampling(data=data_dict)

  print(stan_results)

  results = stan_results.extract()['lambda']
  return results

def plot_stan_simulated_means(concat_mcallister):
  positive, neutral, negative = create_neg_pos_neut(concat_mcallister)

  samples = sts.expon.rvs(scale = 1/0.6, size=(100, 1000))
  means = np.mean(samples, axis=0)
  plt.hist(means, edgecolor = 'black', color = 'mediumseagreen')
  plt.axvline(x = neutral['Engagement'].mean(), color = "purple", linewidth = 4, label="True Mean")
  confint = np.percentile(means, (2.5, 97.5))
  plt.axvline(x = confint[0], color = 'y', ls = "--", linewidth = 4, label = "95% Conf Int")
  plt.axvline(x = confint[1], color = 'y', ls = "--", linewidth = 4)
  plt.xlabel("Simulated Mean")
  plt.ylabel("Frequency")
  plt.legend()

def column_histograms(concat_mcallister):
  positive, neutral, negative = create_neg_pos_neut(concat_mcallister)
  datas = np.asarray([positive['Engagement'], negative['Engagement'], neutral['Engagement']])

  plt.figure(figsize=(5,4))
  for i, j in enumerate(datas):
      plt.plot(sts.uniform.rvs(loc=i+1-0.2, scale=0.4, size=len(j)), 
                j, ',', alpha=0.5)
      
  means = [np.mean(i) for i in datas]

  plt.plot(range(1, 3+1),
          means, 
          marker='_',linewidth=0,color="black",alpha=1,markersize=50)

  plt.xticks(range(1, 3+1), 
            sorted(["Positive", "Neutral", "Negative"]))
  plt.grid(True, alpha=0.25, axis='y')
  plt.ylabel("Number of Engagements")
  plt.xlabel("Comment Sentiment")
  plt.show()

def plot_real_and_infered_expon_dist(concat_mcallister):
  x = np.linspace(0, 12, num=200)
  negative, positive, neutral = create_neg_pos_neut(concat_mcallister)

  plt.figure(figsize=(10, 8))

  plt.hist(neutral['Engagement'], edgecolor = 'black', color='coral', density="True")
  plt.plot(x, sts.expon.pdf(x, scale=1/0.6), label='Infered Distribution')
  plt.xlabel("Number of Engagements")
  plt.ylabel("Frequency")
  plt.legend()

In [ ]:
lst = ['/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/1.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/1.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/2.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/2.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/3.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/3.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/4.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/4.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/5.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/5.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/6.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/7.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/7.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/8.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/8.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/9.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/9.2.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/10.1.csv', '/content/drive/MyDrive/Capstone/McAllister/Slice Analysis/10.2.csv']
concat_mcallister = produce_and_merge(lst)

In [ ]:
positive, neutral, negative = create_pos_neut_neg(concat_mcallister)

In [ ]:
res_pos = {}
res_neg = {}
res_neut = {}

for i in range(14):
  num = sum(positive['Engagement']==i)
  res_pos[i] = num
  num = sum(negative['Engagement']==i)
  res_neg[i] = num
  num = sum(neutral['Engagement']==i)
  res_neut[i] = num

results = pd.DataFrame()
results = results.append(res_pos, ignore_index=True)
results = results.append(res_neg, ignore_index = True)
results = results.append(res_neut, ignore_index=True)
results.index = ['Positive', "Negative", "Neutral"]
results.to_csv("PosNegNeutResults.csv")